In [ ]:
# Pour rouler dans Google Colab, executez cette cellule en premier

!git clone https://github.com/Algolab-Sherhack-2024/mini-defi-grover.git
import sys
sys.path.insert(0,'/content/sherhack_2024')
!pip install -r sherhack_2024/requirements.txt

# Algorithme de Grover

L'algorithme de Grover est souvent présenté comme offrant un avantage quadratique pour effectuer une recherche dans une **base de données non-ordonnée**.

## Apprendre l'algorithme de Grover

Regardez attentivement les deux vidéos sur l'algorithme de Grover dans le cadre des **Énigmes Quantiques**. Prenez des notes si nécessaire.

- Vidéo 1 : [Algorithme de Grover - Énigmes Quantiques](https://www.usherbrooke.ca/iq/enigmesquantiques/#Algo002)
  Cette vidéo introduit Grover avec un circuit très basique (seulement 2 qubits), ce qui le rend facile à reproduire et à comprendre étape par étape.


- Vidéo 2 : [Grover et le problème SAT - Énigmes Quantiques](https://www.usherbrooke.ca/iq/enigmesquantiques/#Algo003) 
  Appliquez l'algorithm Grover mais cette fois avec un problème différent (13 qubits) et de nouvelles notions sur la construction des circuits.

## Défi : Résoudre une nouvelle énigme

Réutilisez le circuit du la video 2 pour résoudre une nouvelle énigme (avec exactement le même nombre de qubits).

Adaptez l'implémentation (le circuit dans le 2eme video) pour résoudre le problème similaire suivant :

![Énigme Image](mini_defi_SAT_puzzle.png)


### Résumé du défi :

**Objectif** : Trouver la bonne combinaison pour les voyants de contrôle (C1, C2, C3, C4), c'est-à-dire l'affectation correcte de 0 (couleur orange) ou 1 (couleur indigo) pour chaque voyant de contrôle.

**But** : Allumer tous les voyants disjoncteurs (D1, D2, ..., D8) avec la valeur 1 (couleur indigo) pour que le voyant principal (p) s'allume également avec la valeur 1 (couleur indigo). Lorsque cela se produit, le coffre s'ouvre.

**Tous les détails du défi sont bien expliqués dans la vidéo 2.**
  

**Conseil :** 

N’hésitez pas à tester différentes configurations, comme le nombre d'itérations de l'algorithme (nombre d'applications de l'oracle et de l'opérateur de diffusion), pour observer l'impact sur les résultats.

---

  
## Fonctions d'Aide pour vos Implémentations

Vous pouvez utiliser les fonctions suivantes dans vos implémentations pour simplifier certaines étapes.



### Importer les bibliothèques nécessaires

In [ ]:
from qiskit import QuantumCircuit
from qiskit import transpile
from qiskit_aer import AerSimulator

from qiskit.visualization import plot_histogram
from qiskit.circuit.library import XGate, ZGate

from typing import Dict

### Instructions de base nécessaires :
- Portes quantiques : NOT (X), Hadamard (H), Z, contrôle-Z (CZ)
- Mesure
- Visualisation

Voici un exemple simple pour illustrer les opérations de base :

In [ ]:
# Générer un circuit simple avec les portes : X, H, Z et CZ

# 1) Créer un nouveau circuit avec deux qubits
circuit = QuantumCircuit(2)

# 2) Ajouter des portes de base
circuit.x(0)  # Ajouter une porte X (NOT) sur le qubit 0
circuit.h(1)  # Ajouter une porte H sur le qubit 1
circuit.z(0)  # Ajouter une porte Z sur le qubit 0
circuit.cz(0, 1)  # Effectuer une porte contrôlée-Z sur le qubit 1, contrôlée par le qubit 0

# 3) Mesurer tous les qubits
circuit.measure_all()  # Mesurer tous les qubits et stocker les résultats sur des bits classiques

# 4) Visualisation
circuit.draw(
    "mpl"
)  # Dessiner le circuit en utilisant Matplotlib ("mpl"). Retirer l'argument "mpl" pour obtenir un dessin textuel.

### L'exécution d'un circuit sur un simulateur

In [10]:
from qiskit_ibm_runtime import SamplerV2 as Sampler


def run_circuit_on_simulator(input_circuit: QuantumCircuit, num_shots: int = 1_000) -> Dict[str, int]:
    """
    Exécute un circuit quantique sur le simulateur Aer en utilisant le primitive Sampler et renvoie les résultats des mesures.

    Arguments:
        input_circuit (QuantumCircuit): Le circuit quantique à simuler
        num_shots (int): Nombre de répétitions de la simulation (par défaut: 1 000)

    Retourne:
        Dict[str, int]: Dictionnaire contenant les résultats des mesures, où:
            - les clés sont les états quantiques mesurés (par ex., '00', '01', '10', '11')
            - les valeurs sont le nombre de fois que chaque état a été mesuré

    Exemple:
        circuit = QuantumCircuit(2)  # Créer un circuit avec 2 qubits
        resultats = run_circuit_(circuit, num_shots=1000)
        # les résultats pourraient ressembler à: {'00': 500, '01': 250, '10': 150, '11': 100}
    """
    # Créer une instance de backend local utilisant AerSimulator
    backend = AerSimulator()

    # Initialiser la primitive Sampler avec le backend local
    # Sampler est une interface de haut niveau pour exécuter des circuits quantiques
    # qui gère automatiquement l'optimisation et l'exécution du circuit
    sampler = Sampler(mode=backend)

    # Exécuter le circuit en utilisant le sampler
    # Le sampler accepte une liste de circuits et renvoie les résultats correspondants
    job = sampler.run([input_circuit], shots=num_shots)

    # Obtenir les résultats du job terminé
    job_result = job.result()

    # Extraire et renvoyer les comptages de mesure des résultats
    # L'index [0] accède aux résultats du premier (et unique) circuit
    counts = job_result[0].data.meas.get_counts()

    return counts

In [11]:
counts = run_circuit_on_simulator(circuit)

### Affichage de l'histogramme des comptes
Afficher les resultat de l'execution 

In [ ]:
plot_histogram(counts)

# Combinaison de deux circuits

In [14]:
def generate_circuit_A(nb_qubits):
    circuit = QuantumCircuit(nb_qubits)
    circuit.x(range(nb_qubits))  # Appliquer la porte X sur tous les qubits
    return circuit


def generate_circuit_B(nb_qubits):
    circuit = QuantumCircuit(nb_qubits)
    circuit.z(range(nb_qubits))  # Appliquer la porte Z sur tous les qubits
    return circuit

In [ ]:
# Circuit principal :
nb_qubits = 2
main_circuit = QuantumCircuit(nb_qubits)
main_circuit.h([0, 1])
main_circuit.barrier()  # une barrière visuelle pour mieux voir le bloc (couche) du circuit

qc_a = generate_circuit_A(nb_qubits)
qc_b = generate_circuit_B(nb_qubits)


main_circuit.append(qc_a, range(nb_qubits))

main_circuit.barrier()

main_circuit.append(qc_b, [0, 1])

main_circuit.measure_all()

main_circuit.draw("mpl")  # ajouter decompose() pour voir plus de détails `main_circuit.decompose().draw('mpl')`

# Porte X multi-contrôle

In [ ]:
total_num_qubits = 4
num_qubits_ctrl = 3
ctrl_state = "011"  # Le bit le plus à droite correspond au qubit 0. len(ctrl_state) == num_qubits_ctrl
qc = QuantumCircuit(total_num_qubits)

# Créer une porte X multi-contrôle avec le bon nombre de qubits
# Le nombre de qubits de contrôle est num_qubits_ctrl
# La porte X sera appliquée sur le qubit après les qubits de contrôle.
mc_xgate = XGate().control(num_ctrl_qubits=num_qubits_ctrl, ctrl_state=ctrl_state)

qc.append(mc_xgate, range(total_num_qubits))

qc.draw("mpl")

In [ ]:
qc.decompose().draw("mpl")